In [ ]:
import pandas as pd
import numpy as np

In [ ]:
closing_price_data = pd.read_pickle("./closing_price_5yr.pkl").dropna(axis = 1)

In [ ]:
stock_names = closing_price_data.keys()

In [ ]:
closing_price_data.head()

In [ ]:
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']
stock_data = closing_price_data[ticket]
stock_data.shape

# Now is for the partitions of data

In [ ]:
class my_mat_funcs():
    import numpy as np
    def mat_mults(*args):
        if(len(args)==1):
            return args[0]
        else:
            return np.matmul(args[0],mat_mults(*args[1:]))

In [ ]:
from collections import OrderedDict
########## Postcondition: calculate each w ##################
def optimal_solution(stock_data_partition, sigma = 'sample', inverse = False):
    
    logret = np.log(stock_data_partition).diff()
    mu = logret[1:].mean()
    if(inverse):
        s, _ = sigma.shape
        one = np.ones((s,1), dtype = float)
        w = 1 / mat_mults(np.transpose(one), sigma, one)[0] * mat_mults(sigma, one)
    else:
        import cvxpy as cvx
        if(sigma == 'sample'):
            sigma = logret.cov()
        s, _ = sigma.shape
        w = cvx.Variable(s)# the length of w is x
        risk = cvx.quad_form(w, sigma.as_matrix())
        prob = cvx.Problem(cvx.Minimize(risk),
                           [cvx.sum_entries(w) == 1])
        prob.solve()
    
    return w.value
    

In [ ]:
#### Postcondition: calculate all w ##############
def Optimization_At_Each_Period(stock_data,days_of_period):
    w=[]
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        stock_data_partition = stock_data[current_period_num*days_of_period:(current_period_num+1)*days_of_period].copy()
        w.append(optimal_solution(stock_data_partition))
    return np.column_stack(w)
#     if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
#         return np.column_stack(w)
#     else:
#         print ("Last Trading Period Not Finished")
#         last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
#         last_end=len(stock_data)-1
#         w.append(optimal_solution(stock_data[last_start:last_end]))
#         return np.column_stack(w)

In [ ]:
#### Postcondition: list of return chunks ################
def return_over_time(stock_data,days_of_period):
    re=[]
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        before = stock_data[(days_of_period*current_period_num+1):(days_of_period*(current_period_num+1)+1)]
        after = stock_data[(days_of_period*current_period_num):(days_of_period*(current_period_num+1))]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)
        
    if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
        return re
    else:
        print ("Last Trading Period Not Finished")
        last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
        last_end=len(stock_data)-1
        before = stock_data[(last_start-1):(last_end-1)]
        after = stock_data[last_start:last_end]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)
        return re

In [ ]:
days_of_period = 252

In [ ]:
def Realized_Return(stock_data,days_of_period, start = '2014-05-21'):
    n = days_of_period
    es = pd.Series(np.arange(n, ))
    Mat_Optimization_At_Each_Period = Optimization_At_Each_Period(stock_data,days_of_period)  # w
    Mat_return_over_time = return_over_time(stock_data,days_of_period)  # return dataframe by chunks
    Total_time = Mat_Optimization_At_Each_Period.shape[1]  # Number of periods
    ss = shortsize(Mat_Optimization_At_Each_Period)  # Size of short side
    daily_r = pd.DataFrame()
    for ticker in range(Total_time):
        tmp = pd.DataFrame(np.matmul(Mat_return_over_time[ticker + 1], Mat_Optimization_At_Each_Period[:,ticker]))
        daily_r  = pd.concat([daily_r, tmp])
    return average_return
    

In [ ]:
np.arange(0,20,6)

In [ ]:
shortsize(Mat_Optimization_At_Each_Period)

In [ ]:
def shortsize(Mat_Optimization_At_Each_Period):
    ss = np.empty(Mat_Optimization_At_Each_Period.shape[1])  # take number of columns (number of periods)
    for i in range(Mat_Optimization_At_Each_Period.shape[1]):
        absw = np.array([np.min(np.array([np.asscalar(j),0])) for j in Mat_Optimization_At_Each_Period[:,i]])
        ss[i] = np.sum(np.abs(absw)) / np.sum(np.abs(Mat_Optimization_At_Each_Period[:,i]))
    return ss

In [ ]:
daily_r = pd.DataFrame()
for ticker in range(Total_time):
    daily_r  = pd.concat([daily_r, pd.DataFrame(np.matmul(Mat_return_over_time[ticker + 1], Mat_Optimization_At_Each_Period[:,ticker]))])

In [ ]:
stock_data.shape

In [ ]:
np.matmul(A,B)

In [ ]:
Mat_return_over_time[ticker].iloc[:,466]

In [ ]:
Mat_Optimization_At_Each_Period.shape

In [ ]:
len(Mat_return_over_time)

In [ ]:
Mat_return_over_time[0].shape

In [ ]:
Mat_Optimization_At_Each_Period.shape

In [ ]:
Mat_return_over_time[0]

In [ ]:
closing_price_data.shape

In [ ]:
ROT=return_over_time(closing_price_data,21)

In [ ]:
ROT.shape

In [ ]:
OMT= Optimization_At_Each_Period(closing_price_data,21)

In [ ]:
Realized_Return(closing_price_data,21)

In [ ]:
OMT.shape

In [ ]:
RR1.shape[1]

In [ ]:
RR1[0,:]

In [ ]:
np.matrix(np.cov(closing_price_data[0:21]))

In [ ]:
np.matrix(np.ones(21))

In [ ]:
np.linalg.inv(np.matmul(
    np.matmul(np.matrix(np.ones(21)),
              np.matrix(np.cov(closing_price_data[0:21]))),
    np.transpose(np.matrix(np.ones(21)))))

In [ ]:
np.linalg.inv(np.matrix(np.ones(21)))

In [ ]:
import cvxpy as cvx

s, _ = sigma.shape

w = cvx.Variable(s)# the length of w is x


risk = cvx.quad_form(w, sigma.as_matrix())
prob = cvx.Problem(cvx.Minimize(risk), 
               [cvx.sum_entries(w) == 1])
prob.solve()
w.value

In [ ]:
closing_price_data[0:1]

In [ ]:
closing_price_data[21:22]

In [ ]:
np.log(np.matrix(closing_price_data[21:22])/np.matrix(closing_price_data[0:1]))

In [ ]:
np.matrix(closing_price_data[21:22])